In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuracoes de Estilo ---
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

# --- Carregamento e Preparacao ---
try:
    # 1. Carrega o arquivo de resultados
    filename = 'vocab_injection_results.csv'
    df = pd.read_csv(filename)
    print(f"Arquivo '{filename}' carregado com sucesso!")

    # 2. Filtra as estrategias para analise
    estrategias_para_analise = ['BaselinePreprocessor', 'RegexPreprocessor', 'Automated_MWE_VocabInjection']
    df_filtrado = df[df['strategy'].isin(estrategias_para_analise)].copy()

    # --- DEFINICAO DE CORES ---
    unique_languages = sorted(df_filtrado['language'].unique())
    colors = sns.color_palette("viridis", n_colors=len(unique_languages))
    language_colors = dict(zip(unique_languages, colors))

    # Separa dataframes para metricas estaticas e dinamicas
    static_df = df_filtrado[df_filtrado['metric_type'] == 'static'].copy()
    dynamic_df = df_filtrado[df_filtrado['metric_type'] == 'dynamic'].dropna(subset=['generation_time']).copy()

    # --- CALCULO DE EFICIENCIA (REDUCAO %) ---
    print("\n" + "="*50)
    print("   RESUMO: ANALISE DE REDUCAO DE TOKENS   ")
    print("="*50)

    # Agrupa e calcula a media de tokens
    summary = static_df.groupby(['language', 'model_or_tokenizer', 'strategy'])['num_tokens'].mean().unstack()

    # Calcula a % de reducao: (Baseline - MWE) / Baseline
    summary['Reducao Real %'] = ((summary['BaselinePreprocessor'] - summary['Automated_MWE_VocabInjection']) / summary['BaselinePreprocessor']) * 100

    # Exibe a tabela de resultados
    print(summary[['BaselinePreprocessor', 'Automated_MWE_VocabInjection', 'Reducao Real %']])
    print("-" * 50)

    # --- GRAFICO 1: ANALISE DE FERTILIDADE ---
    print("\nGerando Grafico 1: Impacto da Expansao de Vocabulario na Fertilidade...")
    fertility_avg = static_df.groupby(['language', 'strategy', 'model_or_tokenizer'])['fertility'].mean().reset_index()

    g = sns.catplot(
        data=fertility_avg,
        x='strategy',
        y='fertility',
        hue='language',
        col='model_or_tokenizer',
        kind='bar',
        height=6,
        aspect=1.1,
        palette=language_colors, # Uso do dicionario consistente
        legend_out=False
    )
    g.fig.suptitle('Impacto da Expansao de Vocabulario na Fertilidade', y=1.03)
    g.set_axis_labels("Estrategia", "Fertilidade (Tokens/Palavra)")
    g.set_titles("Tokenizador: {col_name}")

    # Linha de referencia (Ideal = 1.0)
    for ax in g.axes.flat:
        ax.axhline(1.0, ls='--', color='red', alpha=0.5)
        ax.text(0, 1.02, 'Ideal (1.0)', color='red', fontsize=10)
    
    # Ajuste da legenda
    g.add_legend(title='Lingua')
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    # --- GRAFICO 2: ANALISE DE LATENCIA ---
    print("Gerando Grafico 2: Distribuicao da Latencia (Tempo de Geracao)...")
    plt.figure(figsize=(12, 6))
    
    sns.boxplot(
        data=dynamic_df,
        x='strategy',
        y='generation_time',
        hue='language',
        palette=language_colors, # Uso do dicionario consistente
        showfliers=False
    )
    plt.title('Impacto da Expansao de Vocabulario na Latencia (Gemma-2b)')
    plt.ylabel('Tempo de Geracao (segundos)')
    plt.xlabel('Estrategia')
    plt.legend(title='Lingua')
    plt.tight_layout()
    plt.show()

except FileNotFoundError:
    print(f"ERRO: Nao foi possivel encontrar o arquivo '{filename}'. Verifique o diretorio.")
except Exception as e:
    print(f"Ocorreu um erro durante a execucao: {e}")